<a href="https://colab.research.google.com/github/gen-ai-capstone-project-bartender-agent/MOK-5-ha/blob/main/notebooks/gradio_ui_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing for Kaggle Submission

This notebook is primarily for testing Gradio UI in-notebook. Note that this is not the only valid way in which we can test our use of Gradio, but rather that once we acheieve a desired result in an IDE, we must ensure it can be implemented here as well.

In [1]:
# Remove conflicting packages from the Kaggle base environment.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery

!pip install -q -U "google-genai==1.7.0"

# Install langgraph and the packages used in this notebook.
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

# Installing Gradio to deploy our web app
!pip install -q gradio

Initially testing this notebook on Colab before Kaggle. Code completions and other features make this Jupyter environment a preferred one over developing on Kaggle directly. Due to this, we will see warnings such as above.

In [2]:
#!pip uninstall -qy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
#!pip install -U -q "google-genai==1.7.0"

In [3]:
# Imports

# Data analytics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# UI / Display
import gradio as gr
from IPython.display import Markdown, display
# Gemini - Frontier LLM
from google import genai
from google.genai import types
import os

genai.__version__

'1.7.0'

# Keeping our API Keys Secret



Set up your API key¶
To run the following cell, your API key must be stored it in a Kaggle secret named GOOGLE_API_KEY.

If you don't already have an API key, you can grab one from AI Studio. You can find detailed instructions in the docs.

To make the key available through Kaggle secrets, choose Secrets from the Add-ons menu and follow the instructions to add your key or enable it for this notebook.

In [4]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
class BartendingAgent:
    def __init__(self):
        # Check for required environment variables
        self.gemini_api_key = os.getenv("genai")
        if not self.gemini_api_key:
            raise EnvironmentError(
                "GEMINI_API_KEY not found in environment variables. "
                "Please add it to your .env file."
            )

        try:
            genai.configure(api_key=self.gemini_api_key)
            self.model = genai.GenerativeModel('gemini-2.0-flash')
            logger.info("Successfully initialized Gemini model")
        except Exception as e:
            logger.error(f"Failed to initialize Gemini model: {str(e)}")
            raise RuntimeError(
                f"Failed to initialize Gemini model: {str(e)}. "
                "Please check if your GEMINI_API_KEY is valid."
            )

        self.menu = {
            "1": {"name": "Old Fashioned", "price": 12.00},
            "2": {"name": "Margarita", "price": 10.00},
            "3": {"name": "Mojito", "price": 11.00},
            "4": {"name": "Martini", "price": 13.00},
            "5": {"name": "Whiskey Sour", "price": 11.00},
            "6": {"name": "Gin and Tonic", "price": 9.00},
            "7": {"name": "Manhattan", "price": 12.00},
            "8": {"name": "Daiquiri", "price": 10.00},
            "9": {"name": "Negroni", "price": 11.00},
            "10": {"name": "Cosmopolitan", "price": 12.00}
        }

        self.current_order = []
        self.conversation_history = []

# Bartending_agent.py

In [15]:
from typing import Dict, List, Optional
from langgraph.graph import Graph, StateGraph
from langgraph.prebuilt import ToolNode
import logging
from tenacity import (
    retry,
    stop_after_attempt,
    wait_exponential,
    retry_if_exception_type,
    before_sleep_log
)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class BartendingAgent:
    def __init__(self):
        # Check for required environment variables
        self.model = genai.GenerativeModel('gemini-2.0-flash')
        logger.info("Successfully initialized Gemini model")

        self.menu = {
            "1": {"name": "Old Fashioned", "price": 12.00},
            "2": {"name": "Margarita", "price": 10.00},
            "3": {"name": "Mojito", "price": 11.00},
            "4": {"name": "Martini", "price": 13.00},
            "5": {"name": "Whiskey Sour", "price": 11.00},
            "6": {"name": "Gin and Tonic", "price": 9.00},
            "7": {"name": "Manhattan", "price": 12.00},
            "8": {"name": "Daiquiri", "price": 10.00},
            "9": {"name": "Negroni", "price": 11.00},
            "10": {"name": "Cosmopolitan", "price": 12.00}
        }

        self.current_order = []
        self.conversation_history = []

    def get_menu_text(self) -> str:
        menu_text = "Here's our menu:\n"
        for item_id, item in self.menu.items():
            menu_text += f"{item_id}. {item['name']} - ${item['price']:.2f}\n"
        return menu_text

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=4, max=10),
        retry=retry_if_exception_type((ConnectionError, TimeoutError)),
        before_sleep=before_sleep_log(logger, logging.WARNING),
        reraise=True
    )
    def process_order(self, text: str) -> str:
        """Processes user input using the Gemini model for conversational interaction."""
        try:
            # Add the user's input to conversation history
            self.conversation_history.append({"role": "user", "parts": [text]})
            logger.info(f"Processing user input: {text}")

            # --- Construct the prompt for Gemini ---
            prompt_parts = [
                "You are a friendly and helpful bartender taking drink orders.",
                "Be conversational. Ask clarifying questions if the order is unclear.",
                "If the user asks for something not on the menu, politely tell them and show the menu again.",
                "If the user asks to see their current order, list the items and their prices.",
                "\nHere is the menu:",
                self.get_menu_text(),
                "\nCurrent order:",
            ]
            if self.current_order:
                order_text = "\n".join([f"- {item['name']} (${item['price']:.2f})" for item in self.current_order])
                prompt_parts.append(order_text)
            else:
                prompt_parts.append("No items ordered yet.")

            prompt_parts.append("\nConversation History:")
            # Add previous turns, limiting history length if necessary
            history_limit = 10 # Keep the last 10 turns
            limited_history = self.conversation_history[-(history_limit + 1):-1] # Exclude the latest user input already added

            for entry in limited_history:
                 # Ensure history parts are strings. Handle potential list/dict structures if they exist.
                 content = entry.get("parts", [""])[0] # Assuming parts is a list with one string
                 if isinstance(content, dict): # Handle cases where parts might be structured differently
                     content = str(content) # Fallback to string representation
                 role = entry.get("role", "unknown")
                 prompt_parts.append(f"{role.capitalize()}: {content}")


            prompt_parts.append(f"\nUser: {text}")
            prompt_parts.append("\nBartender:")

            full_prompt = "\n".join(prompt_parts)
            logger.debug(f"Full prompt for Gemini:\n{full_prompt}")

            # --- Call the Gemini model ---
            # Use stream=False for a single response object
            # Safety settings can be configured here if needed
            generation_config = genai.types.GenerationConfig(
                # candidate_count=1, # Default is 1
                # stop_sequences=None,
                # max_output_tokens=2048, # Already set in .env? Agent could load this.
                temperature=0.7 # Already set in .env? Agent could load this.
            )

            response = self.model.generate_content(
                contents=[full_prompt], # Send as a list for single-turn
                generation_config=generation_config,
                # safety_settings='HARM_BLOCK_THRESHOLD_UNSPECIFIED' # Adjust safety if needed
            )

            # --- Process the response ---
            if not response.candidates or not response.candidates[0].content.parts:
                 logger.error("Gemini response was empty or invalid.")
                 agent_response_text = "Sorry, I had trouble understanding that. Could you please rephrase?"
            else:
                 agent_response_text = response.candidates[0].content.parts[0].text
                 logger.info(f"Gemini response: {agent_response_text}")

                 # Simple check if the *model's response* mentions adding an item
                 # This is basic; a more robust approach might involve asking the model
                 # to output structured data or using function calling.
                 for item_id, item in self.menu.items():
                     if item["name"].lower() in agent_response_text.lower() and \
                        any(add_word in agent_response_text.lower() for add_word in ["added", "adding", "got it", "sure thing"]):
                          # Avoid adding duplicates if already in order from this turn?
                          # This logic might need refinement based on model behavior.
                          if not self.current_order or item["name"] != self.current_order[-1]["name"]:
                              self.current_order.append(item)
                              logger.info(f"Added '{item['name']}' to order based on Gemini response.")
                              # Only add the first match found in the response?
                              break

            # Add the agent's response to conversation history
            self.conversation_history.append({"role": "assistant", "parts": [agent_response_text]})

            return agent_response_text

        # except genai.types.BlockedPromptError as e: # Requires specific import if needed
        #     logger.error(f"Gemini prompt blocked: {e}")
        #     return "I'm sorry, I can't respond to that request due to safety guidelines."
        # except genai.types.StopCandidateException as e: # Requires specific import if needed
        #     logger.error(f"Gemini response stopped: {e}")
        #     # The partial response might be in e.response
        #     return response.candidates[0].content.parts[0].text if response and response.candidates else "My response was cut short."
        except Exception as e:
            logger.exception(f"Error processing order with Gemini: {str(e)}") # Use logger.exception for traceback
            # Fallback response
            return "I'm sorry, there was an internal error processing your request. Please try again."

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=4, max=10),
        retry=retry_if_exception_type((ConnectionError, TimeoutError)),
        before_sleep=before_sleep_log(logger, logging.WARNING),
        reraise=True
    )
    #def get_voice_response(self, text: str) -> str:
        #try:
            #if not self.cartesia_api_key:
                #raise RuntimeError(
                    #"Cannot generate voice response: CARTESIA_API_KEY is not set"
                #)

            #logger.info(f"Generating voice response for text: {text}")

            # This would be where you'd call the Cartesia API
            # For now, we'll just return the text
            #return text
        #except Exception as e:
            #logger.error(f"Error generating voice response: {str(e)}")
            #raise RuntimeError(
                #f"Failed to generate voice response: {str(e)}. "
                #"Please check your CARTESIA_API_KEY and network connection."
            #)

    def reset_order(self):
        try:
            self.current_order = []
            self.conversation_history = []
            logger.info("Order and conversation history reset successfully")
        except Exception as e:
            logger.error(f"Error resetting order: {str(e)}")
            raise RuntimeError(f"Failed to reset order: {str(e)}")

# Main.py

In [16]:
import logging

# Configure logging (optional but recommended)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize the Bartending Agent
try:
    agent = BartendingAgent()
    logger.info("Bartending Agent initialized successfully.")
except EnvironmentError as e:
    logger.error(f"EnvironmentError initializing agent: {e}")
    # Exit or provide a dummy agent if API keys are missing
    # For now, we'll re-raise to make the issue clear
    raise
except RuntimeError as e:
    logger.error(f"RuntimeError initializing agent: {e}")
    # Handle other initialization errors (e.g., Gemini connection)
    raise

# Define the function that Gradio will call
def handle_order(user_input, chat_history):
    logger.info(f"Received user input: {user_input}")

    # Process the order using the agent
    response_text = agent.process_order(user_input)
    logger.info(f"Agent response text: {response_text}")

    # Optionally, generate voice response (if Cartesia API is configured and needed)
    # voice_response = agent.get_voice_response(response_text)
    # logger.info(f"Agent voice response generated (placeholder): {voice_response}")

    # Update chat history
    chat_history.append((user_input, response_text))

    # Return updated chat history and potentially the voice response
    # For now, just returning text response updates
    return "", chat_history # Return empty string to clear input box

# Define the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Bartending Agent")
    gr.Markdown(agent.get_menu_text()) # Display the menu

    chatbot = gr.Chatbot(label="Conversation", value=[]) # Initialize chatbot display

    msg = gr.Textbox(label="Your Order", placeholder="What can I get for you?")

    clear = gr.Button("Clear Conversation")

    msg.submit(handle_order, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: (agent.reset_order(), []), None, [chatbot], queue=False)

# Launch the Gradio interface
if __name__ == "__main__":
    logger.info("Launching Gradio interface...")
    # Set share=True to create a public link (optional)
    demo.launch(share=True)
    logger.info("Gradio interface launched.")

<ipython-input-16-88977ad0fd75>:45: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", value=[]) # Initialize chatbot display


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e54c4e39d858b0167e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
